<a href="https://colab.research.google.com/github/Ajay-user/Apache-beam/blob/main/foundation/Reading_from_a_SQLite_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ! pip install apache-beam -q

## Create SQLite local database file

### Connection

In [34]:
import sqlite3

db_name = 'moon-phase.db'

### Create Table

In [35]:
with sqlite3.connect(db_name) as db:
  cursor = db.cursor()
  query = """
          create table if not exists moonphases (
            id integer primary key,
            phase_emoji text not null,
            peak_datetime datetime not null,
            phase text not null
          )
  """

  cursor.execute(query)

### Insert data

In [36]:
with sqlite3.connect(db_name) as db:
  cursor = db.cursor()
  query = """
      insert into moonphases (phase_emoji, peak_datetime, phase) values (?,?,?)
  """

  cursor.executemany(query, [
    ('🌕', '2017-12-03 15:47:00', 'Full Moon'),
    ('🌗', '2017-12-10 07:51:00', 'Last Quarter'),
    ('🌑', '2017-12-18 06:30:00', 'New Moon'),
    ('🌓', '2017-12-26 09:20:00', 'First Quarter'),
    ('🌕', '2018-01-02 02:24:00', 'Full Moon'),
    ('🌗', '2018-01-08 22:25:00', 'Last Quarter'),
    ('🌑', '2018-01-17 02:17:00', 'New Moon'),
    ('🌓', '2018-01-24 22:20:00', 'First Quarter'),
    ('🌕', '2018-01-31 13:27:00', 'Full Moon'),

  ])

### Select from db


In [37]:
with sqlite3.connect(db_name) as db:
  cursor = db.cursor()
  query = """
      select * from moonphases;
  """

  cursor.execute(query)

  for row in cursor.fetchall():
    print(row)

(1, '🌕', '2017-12-03 15:47:00', 'Full Moon')
(2, '🌗', '2017-12-10 07:51:00', 'Last Quarter')
(3, '🌑', '2017-12-18 06:30:00', 'New Moon')
(4, '🌓', '2017-12-26 09:20:00', 'First Quarter')
(5, '🌕', '2018-01-02 02:24:00', 'Full Moon')
(6, '🌗', '2018-01-08 22:25:00', 'Last Quarter')
(7, '🌑', '2018-01-17 02:17:00', 'New Moon')
(8, '🌓', '2018-01-24 22:20:00', 'First Quarter')
(9, '🌕', '2018-01-31 13:27:00', 'Full Moon')


We could use a `FlatMap` transform to receive a SQL query and `yield` each result row, but that would mean creating a new database connection for each query. If we generated a large number of queries, creating that many connections could be a bottleneck.

It would be nice to create the database connection only once for each worker, and every query could use the same connection if needed.

We can use a
[custom `DoFn` transform](https://beam.apache.org/documentation/transforms/python/elementwise/pardo/#example-3-pardo-with-dofn-methods)
for this. It allows us to open and close resources, like the database connection, only _once_ per `DoFn` _instance_ by using the `setup` and `teardown` methods.

> ℹ️ It should be safe to _read_ from a database with multiple concurrent processes using the same connection, but only one process should be _writing_ at once.

In [33]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from typing import List, Dict, Tuple, Iterable

In [39]:
class SQLiteSelect(beam.DoFn):
  
  def __init__(self, database_file: str):
    self.database_file = database_file

  def setup(self):
    self.connection = sqlite3.connect(self.database_file)

  def process(self, query: Tuple[str, List[str]]) -> Iterable[Dict[str, str]]:
    table, columns = query
    cursor = self.connection.cursor()
    cursor.execute(f"SELECT {','.join(columns)} FROM {table}")
    for row in cursor.fetchall():
      yield dict(zip(columns, row))

  def teardown(self):
    self.connection.close()

In [40]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.pvalue.PBegin)
@beam.typehints.with_output_types(Dict[str, str])
def SelectFromSQLite(
    pbegin: beam.pvalue.PBegin,
    database_file: str,
    queries: List[Tuple[str, List[str]]],
) -> beam.PCollection[Dict[str, str]]:

  return (
      pbegin
      | 'Create inputs' >> beam.Create(queries)
      | 'SQLite SELECT' >> beam.ParDo(SQLiteSelect(database_file))
  )

In [41]:

queries = [
    # (table_name, [column1, column2, ...])
    ('moonphases', ['phase_emoji', 'peak_datetime', 'phase']),
    ('moonphases', ['phase_emoji', 'phase']),
]


options = PipelineOptions(flags=[], type_check_additional='all')

with beam.Pipeline(options=options) as pipeline:
  (
      pipeline
      | 'Read from SQLite' >> SelectFromSQLite(db_name, queries)
      | 'Print rows' >> beam.Map(print)
  )

{'phase_emoji': '🌕', 'peak_datetime': '2017-12-03 15:47:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2017-12-10 07:51:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2017-12-18 06:30:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2017-12-26 09:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-02 02:24:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'peak_datetime': '2018-01-08 22:25:00', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'peak_datetime': '2018-01-17 02:17:00', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'peak_datetime': '2018-01-24 22:20:00', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-31 13:27:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗', 'phase': 'Last Quarter'}
{'phase_emoji': '🌑', 'phase': 'New Moon'}
{'phase_emoji': '🌓', 'phase': 'First Quarter'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌗',